In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dyfi_earthquake_data.csv")
print(df.head())

                       time   latitude   longitude  depth   mag magType   nst  \
0  2025-01-17T08:55:51.812Z   8.424500  125.994900  10.00  5.00      mb  44.0   
1  2025-01-17T08:16:10.130Z  37.361168 -121.717499   8.24  2.78      md  79.0   
2  2025-01-17T04:07:30.060Z  40.292667 -124.461502  19.50  2.78      md  24.0   
3  2025-01-17T01:51:09.250Z  42.189500 -124.380000  -0.58  2.21      ml  16.0   
4  2025-01-17T01:46:17.623Z   9.403000   40.070400  10.00  4.70      mb  39.0   

     gap     dmin   rms  ...                   updated  \
0  113.0  1.40800  1.05  ...  2025-01-17T09:59:42.345Z   
1   29.0  0.10800  0.09  ...  2025-01-17T09:32:18.040Z   
2  258.0  0.09106  0.08  ...  2025-01-17T04:32:15.491Z   
3  205.0  0.04854  0.23  ...  2025-01-17T03:58:41.101Z   
4  138.0  6.64100  0.63  ...  2025-01-17T09:44:38.530Z   

                                place        type horizontalError depthError  \
0  1 km ENE of Lapinigan, Philippines  earthquake           10.16      1.845   
1   

In [ ]:
required_columns = ['latitude', 'longitude', 'depth', 'mag', 'nst', 'gap', 'dmin', 'rms']
df = df[required_columns].dropna()
df['feel_percentage'] = np.clip(10 * (df['mag'] - 3), 0, 100)

print(df.head())

    latitude   longitude  depth   mag   nst    gap     dmin   rms  \
0   8.424500  125.994900  10.00  5.00  44.0  113.0  1.40800  1.05   
1  37.361168 -121.717499   8.24  2.78  79.0   29.0  0.10800  0.09   
2  40.292667 -124.461502  19.50  2.78  24.0  258.0  0.09106  0.08   
3  42.189500 -124.380000  -0.58  2.21  16.0  205.0  0.04854  0.23   
4   9.403000   40.070400  10.00  4.70  39.0  138.0  6.64100  0.63   

   feel_percentage  
0             20.0  
1              0.0  
2              0.0  
3              0.0  
4             17.0  


In [ ]:
X = df[['latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms']].values
y = df['feel_percentage'].values

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

print("Data split complete. Ready for training.")


Data split complete. Ready for training.


In [ ]:
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")


Model Performance:
Mean Absolute Error: 1.67
R² Score: 0.89


In [ ]:
lat = float(input("Enter Latitude (.....): "))
lon = float(input("Enter Longitude (.....): "))
depth = float(input("Enter Depth (km): "))

nst_avg = df['nst'].mean()
gap_avg = df['gap'].mean()
dmin_avg = df['dmin'].mean()
rms_avg = df['rms'].mean()

InputX = np.array([[lat, lon, depth, nst_avg, gap_avg, dmin_avg, rms_avg]])
InputX_norm = scaler.transform(InputX)

predicted_feel = gb_model.predict(InputX_norm)
print(f"Predicted 'In-Person Feel Percentage': {predicted_feel[0]:.2f}%")


Enter Latitude (.....): 8.425
Enter Longitude (.....): 125.995
Enter Depth (km): 10
Predicted 'In-Person Feel Percentage': 16.59%


In [ ]:
import numpy as np
def get_coefficients(lat, lon):
    if 24 <= lat <= 46 and 122 <= lon <= 153:
        return 1.6, 0.6, 0.02
    elif -10 <= lat <= 10 and 100 <= lon <= 120:
        return 1.2, 0.8, 0.03
    elif 20 <= lat <= 40 and 60 <= lon <= 80:
        return 1.3, 0.75, 0.025
    elif 30 <= lat <= 50 and -130 <= lon <= -100:
        return 1.5, 0.7, 0.02
    elif 10 <= lat <= 40 and -100 <= lon <= -80:
        return 1.4, 0.65, 0.025
    elif -60 <= lat <= -10 and -80 <= lon <= -30:
        return 1.5, 0.6, 0.03
    elif -40 <= lat <= -10 and 110 <= lon <= 155:
        return 1.3, 0.7, 0.02
    elif -35 <= lat <= -15 and 15 <= lon <= 35:
        return 1.4, 0.8, 0.035
    elif 35 <= lat <= 55 and -10 <= lon <= 40:
        return 1.2, 0.85, 0.03
    elif 55 <= lat <= 75 and -170 <= lon <= -50:
        return 1.1, 0.9, 0.04
    elif -75 <= lat <= -60 and -180 <= lon <= 180:
        return 1.0, 0.95, 0.05
    else:
        return 1.2, 0.8, 0.03


def calculate_mmi(lat, lon, depth, magnitude):
    a, b, c = get_coefficients(lat, lon)
    d = 1.5
    mmi = a * magnitude - b * np.log10(1) - c * depth + d
    return np.clip(mmi, 1, 12)


def predict_in_person_feel(mmi):
    if mmi < 3:
        return 0.0
    elif 3 <= mmi < 5:
        return 30.0
    elif 5 <= mmi < 7:
        return 70.0
    elif 7 <= mmi < 9:
        return 90.0
    else:
        return 99.0


lat = float(input("Enter Latitude: "))
lon = float(input("Enter Longitude: "))
depth = float(input("Enter Depth (km): "))
magnitude = float(input("Enter Magnitude: "))


mmi = calculate_mmi(lat, lon, depth, magnitude)
in_person_feel = predict_in_person_feel(mmi)


print(f"\nCalculated MMI: {mmi:.2f}")
print(f"Predicted In-Person Feel: {in_person_feel:.1f}%")


Enter Latitude: 30
Enter Longitude: 30
Enter Depth (km): 30
Enter Magnitude: 1

Calculated MMI: 1.80
Predicted In-Person Feel: 0.0%
